In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import checklist
import spacy
import itertools

In [404]:
import checklist.editor
import checklist.text_generation
from checklist.mft import Mft
from checklist.inv_dir import Inv, Dir
from checklist.expect import Expect

In [8]:

nlp = spacy.load('en_core_web_sm')

In [9]:
tg = checklist.text_generation.TextGenerator(nlp)

In [35]:
from checklist.pred_wrapper import PredictorWrapper

In [161]:
import sys
sys.path.append('/home/marcotcr/work/ml-tests/')
from mltests import model_wrapper
sentiment = model_wrapper.ModelWrapper()
pp = PredictorWrapper.wrap_softmax(sentiment.predict_proba)

In [10]:
# tg.unmask('I don\'t know <mask> I am going to cope.')
[x[0][0] for x in tg.unmask('I really <mask> the pilot.')]

['liked',
 'like',
 'enjoyed',
 'loved',
 'admired',
 'love',
 'admire',
 'enjoy',
 'appreciated',
 'appreciate',
 'respect',
 'was',
 'disliked',
 'miss',
 'respected',
 'likes',
 'trust',
 'missed',
 'hated',
 'wanted']

In [11]:
a = tg.unmask('I really <mask> the pilot.')
b = tg.unmask('I really <mask> the flight.')

In [12]:
# tg.unmask_multiple(['I really <mask> the pilot.', 'I really <mask> the flight.'], metric='min')

In [202]:
editor = checklist.editor.Editor()
editor.lexicons['male'] = ['John', 'Luke', "Mark"]
editor.lexicons['female'] = ['Mary', 'Judy', "Julia"]

In [15]:
nouns = ['flight', 'seat', 'pilot', 'staff', 'plane', 'airline', 'cabin crew', 'aircraft', 'food']
pos = ['liked', 'enjoyed', 'loved', 'admired', 'appreciated']
neg = ['hated', 'disliked', 'regretted']


In [16]:
ts = editor.template('I really <mask> the {n}.', n=nouns)
print([x[0][0] for x in tg.unmask_multiple(ts, beam_size=1000)][:20])
# tg.unmask_multiple(ts, metric='avg', beam_size=100)

['like', 'liked', 'love', 'enjoyed', 'loved', 'appreciate', 'enjoy', 'appreciated', 'miss', 'Like', 'missed', 'needed', 'hate', 'wanted', 'likes', 'got', 'admired', 'admire', 'hated', 'recommend']


In [17]:
ts = editor.template('<mask> really {pn} the {n}.', n=nouns, pn=pos+neg)
print([x[0][0] for x in tg.unmask_multiple(ts, beam_size=1000)][:20])
# tg.unmask_multiple(ts, metric='avg', beam_size=100)

['I', 'We', 'He', 'he', 'She', 'They', 'we', 'and', '...', 'she', 'they', 'Everyone', 'i', 'People', 'You', 'Alex', 'Everybody', 'And', 'Dad', 'John']


In [18]:
subj = ['I', 'We', 'They', 'we', 'He', 'he', 'She', 'she', 'they', 'people', 'People', 'you', '']

In [19]:
ts = editor.template('I didn\'t like the {n}, it was very <mask>.', n=nouns)
print([x[0][0] for x in tg.unmask_multiple(ts, beam_size=1000, metric='avg')][:100])
# tg.unmask_multiple(ts, metric='avg', beam_size=100)

['bad', 'boring', 'strange', 'cold', 'slow', 'unpleasant', 'weird', 'difficult', 'uncomfortable', 'expensive', 'rough', 'noisy', 'ugly', 'poor', 'dark', 'different', 'small', 'disappointing', 'quiet', 'rude', 'confusing', 'dull', 'sad', 'odd', 'loud', 'annoying', 'simple', 'harsh', 'short', 'unusual', 'scary', 'hard', 'nice', 'hot', 'dangerous', 'aggressive', 'heavy', 'dirty', 'conservative', 'old', 'tough', 'bland', 'low', 'cheap', 'frustrating', 'depressing', 'nasty', 'long', 'crude', 'horrible', 'stupid', 'complicated', 'funny', 'fast', 'good', 'basic', 'weak', 'wrong', 'awkward', 'terrible', 'negative', 'crowded', 'plain', 'thin', 'interesting', 'awful', 'bizarre', 'intimidating', 'cool', 'primitive', 'unstable', 'flat', 'narrow', 'stressful', 'stiff', 'empty', 'chaotic', 'light', 'irritating', 'big', 'unfair', 'sick', 'similar', 'angry', 'mean', 'predictable', 'easy', 'offensive', 'foreign', 'violent', 'tight', 'comfortable', 'busy', 'challenging', 'arrogant', 'disgusting', 'ineff

In [20]:
pos_adj = ['good', 'nice', 'helpful', 'comfortable', 'cool', 'efficient', 'pleasant', 'interesting', 'impressive', 'welcoming', 'professional', 'beautiful', 'exciting', 'positive', 'solid', 'amazing', 'wonderful', 'lovely']
neg_adj = ['bad', 'boring', 'unpleasant', 'difficult', 'uncomfortable', 'ugly', 'poor', 'disappointing', 'sad', 'annoying', 'dirty', 'frustrating', 'depressing', 'nasty', 'horrible', 'stupid', 'negative', 'awful', 'stressful', 'irritating', 'disgusting']

In [21]:
pos_eg = editor.template('{sub}{really} {p} the {n}.', sub=subj, really=[' really', ''], n=nouns, p=pos)
pos_eg += editor.template('The {n} was{very} {p}.', sub=subj, very=[' very', ''], n=nouns, p=pos_adj)
neg_eg = editor.template('{sub}{really} {p} the {n}.', sub=subj, really=[' really', ''], n=nouns, p=neg)
neg_eg += editor.template('The {n} was{very} {p}.', sub=subj, very=[' very', ''], n=nouns, p=neg_adj)

In [185]:
len(pos_eg), len(neg_eg)

(1494, 1080)

In [255]:
test_pos = Mft(pos_eg, labels=1)
test_neg = Mft(neg_eg, labels=0)

In [187]:
test_pos.run(pp)
test_neg.run(pp)

In [190]:
test_pos.results.passed.mean(), test_neg.results.passed.mean()

(1.0, 0.9833333333333333)

In [199]:
for f in np.where(test.results.passed == False)[0]:
    print(test.results.confs[f][1].round(3), test.data[f])

0.999 The flight was very stressful.
0.995 The seat was very stressful.
0.999 The pilot was very stressful.
1.0 The staff was very stressful.
0.999 The plane was very stressful.
0.998 The airline was very stressful.
0.998 The cabin crew was very stressful.
0.999 The aircraft was very stressful.
0.999 The food was very stressful.
0.999 The flight was stressful.
0.963 The seat was stressful.
0.997 The pilot was stressful.
0.999 The staff was stressful.
0.999 The plane was stressful.
0.97 The airline was stressful.
0.994 The cabin crew was stressful.
0.998 The aircraft was stressful.
0.997 The food was stressful.


## Inv, dir

In [200]:
import csv
r = csv.DictReader(open('/home/marcotcr/datasets/airline/Tweets.csv'))
labels = []
confs = []
airlines = []
tdata = []
reasons = []
for row in r:
    sentiment, conf, airline, text = row['airline_sentiment'], row['airline_sentiment_confidence'], row['airline'], row['text']
    labels.append(sentiment)
    confs.append(conf)
    airlines.append(airline)
    tdata.append(text)
    reasons.append(row['negativereason'])

mapping = {'negative': 0, 'positive': 2, 'neutral': 1}
labels = np.array([mapping[x] for x in labels]).astype(int)

In [201]:
sentences = tdata
parsed_data = list(nlp.pipe(sentences))

In [240]:
def add_typo(string):
    string = list(string)
    swaps = 1
    swaps = np.random.choice(len(string) - 1, swaps)
    for swap in swaps:
        swap = np.random.choice(len(string) - 1)
        tmp = string[swap]
        string[swap] = string[swap + 1]
        string[swap + 1] = tmp
    return ''.join(string)

def add_typos(string):
    return list(set([add_typo(string) for _ in range(10)]))


In [233]:
from checklist.perturb import Perturb

In [400]:
data = Perturb.perturb(np.random.choice(sentences, 100), add_typos, keep_original=True)

In [401]:
test = Inv(data, threshold=0.1)
test.run(pp, overwrite=True)

In [402]:
test.results.passed.mean()

0.77

In [403]:
for f in np.where(test.results.passed == False)[0][:5]:
    dataconfs = list(zip(test.data[f], test.results.confs[f][:, 1]))
    dataconfs[1:] = sorted(dataconfs[1:], key=lambda x: np.abs(x[1] - dataconfs[0][1]), reverse=True)
    print('\n'.join(['%.2f %s' % (x[1], x[0]) for x in dataconfs][:3]))
    print()

0.01 @USAirways I hope flight 1855 is not being delayed because flight attendant didn't come to work! #nobackup
0.96 @USAirways Ih ope flight 1855 is not being delayed because flight attendant didn't come to work! #nobackup
0.00 @USAirways I hope flight 1855 is not beingd elayed because flight attendant didn't come to work! #nobackup

0.01 @JetBlue's CEO battles to appease passengers and Wall Street - Waterbury Republican American http://t.co/Of7PFvQpOY
0.95 @JetBlue's CEOb attles to appease passengers and Wall Street - Waterbury Republican American http://t.co/Of7PFvQpOY
0.91 @JetBlue's CEO battlest o appease passengers and Wall Street - Waterbury Republican American http://t.co/Of7PFvQpOY

0.95 @JetBlue ETA for flight 802 into Orlando this morning
0.04 @JetBlue ETA for flight 802 into Orlando this mornnig
0.29 @eJtBlue ETA for flight 802 into Orlando this morning

0.27 @united I think you should board from the back of the plane #whatstheholdup #CHItoCLE
0.92 @united I think you shoul

In [399]:
data = Perturb.perturb(np.random.choice(sentences, 1000), add_typo, keep_original=True)
test = Inv(data, threshold=0.1)
test.run(pp, overwrite=True)

In [395]:
test.results.passed.mean()

0.954

In [396]:
for f in np.where(test.results.passed == False)[0][:5]:
    dataconfs = list(zip(test.data[f], test.results.confs[f][:, 1]))
    dataconfs[1:] = sorted(dataconfs[1:], key=lambda x: np.abs(x[1] - dataconfs[0][1]), reverse=True)
    print('\n'.join(['%.2f %s' % (x[1], x[0]) for x in dataconfs][:3]))
    print()

0.42 @AmericanAir it wasn't ' disrupted' it was Cancelled Flightled. Airport agents were horrendous. Sharon was your saviour
0.99 @AmericanAir it wasn't ' disrupted' it was Cancelled Flightled. Airport agents were horrenduos. Sharon was your saviour

0.80 @JetBlue gr8 #Mint crew on #flight 123 to #LAX they're #Mintalicious #TrueBlueLove #ShelleyandMarcRock #travel #air
0.20 @JetBlue gr8 #Mint crew on #flight 123 to #LAX theyr'e #Mintalicious #TrueBlueLove #ShelleyandMarcRock #travel #air

0.16 Need more info on that! @united: @momsgoodeats We have great amenity kits. PJ's not included. We status match AA. ^KP”
0.85 Need more info on that! @united: @momsgoodeats We have great amenity kits. P'Js not included. We status match AA. ^KP”

0.08 @AmericanAir can you guys help me please?
0.78 @AmericanAir can you guys help em please?

0.10 @USAirways I finally got through on the phone after a 2 hour wait time.
0.55 @USAirways I finally got through on th ephone after a 2 hour wait time.



In [433]:
# TODO: get from somethwere else
def add_negatives(string):
    string = string.strip('.')
    return [string + '. ' + l for l in ['I hate you', 'I despise you', 'You suck']]
def add_positive(string):
    string = string.strip('.')
    return [string + '. ' + l for l in ['I love you', 'I like you', 'You are great']]


In [412]:
add_negatives(sentences[0])

['@VirginAmerica What @dhepburn said. I hate you',
 '@VirginAmerica What @dhepburn said. I despise you',
 '@VirginAmerica What @dhepburn said. You suck']

In [441]:
data = Perturb.perturb(np.random.choice(sentences, 1000), add_negatives, keep_original=True)
test = Dir(data, Expect.wrap(Expect.all(Expect.pairwise_to_group(Expect.monotonic_label(1, increasing=False, tolerance=0)))) )
test.run(pp, overwrite=True)
test.results.passed.mean()

0.893

In [442]:
for f in np.where(test.results.passed == False)[0][:5]:
    dataconfs = list(zip(test.data[f], test.results.confs[f][:, 1]))
    dataconfs[1:] = sorted(dataconfs[1:], key=lambda x: (x[1] - dataconfs[0][1]), reverse=True)
    print('\n'.join(['%.5f %s' % (x[1], x[0]) for x in dataconfs][:3]))
    print()

0.00039 @united It's taken ~3 weeks for Cust Care to respond to Case #8477733, requesting a refund for wi-fi not working. Where can I get an update?
0.00044 @united It's taken ~3 weeks for Cust Care to respond to Case #8477733, requesting a refund for wi-fi not working. Where can I get an update?. I despise you
0.00035 @united It's taken ~3 weeks for Cust Care to respond to Case #8477733, requesting a refund for wi-fi not working. Where can I get an update?. I hate you

0.00040 @AmericanAir why was my flight from Miami to Dallas Monday  rescheduled to Tuesday?
0.00047 @AmericanAir why was my flight from Miami to Dallas Monday  rescheduled to Tuesday?. I despise you
0.00040 @AmericanAir why was my flight from Miami to Dallas Monday  rescheduled to Tuesday?. You suck

0.00179 @united delayed another hour
0.00212 @united delayed another hour. You suck
0.00067 @united delayed another hour. I despise you

0.00057 @united aaaand 7 hours Late Flightr won't be catching a flight til 6AM...stuck

In [443]:
data = Perturb.perturb(np.random.choice(sentences, 1000), add_positive, keep_original=True)
test = Dir(data, Expect.wrap(Expect.all(Expect.pairwise_to_group(Expect.monotonic_label(1, increasing=True, tolerance=0)))) )
test.run(pp, overwrite=True)
test.results.passed.mean()

0.989

In [444]:
for f in np.where(test.results.passed == False)[0][:5]:
    dataconfs = list(zip(test.data[f], test.results.confs[f][:, 1]))
    dataconfs[1:] = sorted(dataconfs[1:], key=lambda x: (- x[1] + dataconfs[0][1]), reverse=True)
    print('\n'.join(['%.5f %s' % (x[1], x[0]) for x in dataconfs][:3]))
    print()

0.00042 @USAirways 1500 characters are not enough to convey the issue. Was directed to emailmailto:customer.relations@usairways.com unmonitored. BAD
0.00038 @USAirways 1500 characters are not enough to convey the issue. Was directed to emailmailto:customer.relations@usairways.com unmonitored. BAD. I love you
0.00345 @USAirways 1500 characters are not enough to convey the issue. Was directed to emailmailto:customer.relations@usairways.com unmonitored. BAD. I like you

0.99994 @JetBlue #Bluemanity @AirlineAdviser loves this. Have a great time flying this
0.99993 @JetBlue #Bluemanity @AirlineAdviser loves this. Have a great time flying this. I love you
0.99993 @JetBlue #Bluemanity @AirlineAdviser loves this. Have a great time flying this. I like you

0.99993 @SouthwestAir oh, ok!  all good!  looking forward to escaping the cold for a bit!  thanks!
0.99993 @SouthwestAir oh, ok!  all good!  looking forward to escaping the cold for a bit!  thanks!. I love you
0.99993 @SouthwestAir oh, ok!  a

### QQP

In [448]:
qqp = model_wrapper.ModelWrapper()
pp = PredictorWrapper.wrap_softmax(qqp.predict_proba)

In [452]:
qs = []
labels = []
all_questions = set()
for x in open('/home/marcotcr/datasets/glue/glue_data/QQP/dev.tsv').readlines()[1:]:
    try:
        q1, q2, label = x.strip().split('\t')[3:]
    except:
        print(x)
        continue
    all_questions.add(q1)
    all_questions.add(q2)
    qs.append((q1, q2))
    labels.append(label)
labels = np.array(labels).astype(int)

"	What does the Quran say about homosexuality?	0



In [455]:
quran = [x[0][0] for x in tg.unmask('What does the Quran say about <mask>?')]

In [461]:
quran.remove('homosexuals')
quran.remove('gays')
quran.remove('this')

In [463]:
data = editor.template(('What does the Quran say about {thing1}?', 'What does the Quran say about {thing2}?'),
                thing1=quran, thing2=quran, remove_duplicates=True)
test = Mft(data, labels=0)
test.run(pp, overwrite=True)
test.results.passed.mean()

In [474]:
for f in np.where(test.results.passed == False)[0]:
    print(test.results.confs[f][1].round(3), test.data[f])

0.581 ('What does the Quran say about polygamy?', 'What does the Quran say about circumcision?')
0.974 ('What does the Quran say about Islam?', 'What does the Quran say about Muslims?')
0.817 ('What does the Quran say about Muhammad?', 'What does the Quran say about Muslims?')
0.845 ('What does the Quran say about Muslims?', 'What does the Quran say about Islam?')
0.717 ('What does the Quran say about circumcision?', 'What does the Quran say about polygamy?')
0.956 ('What does the Quran say about marriage?', 'What does the Quran say about polygamy?')


In [479]:
print([x[0][0] for x in tg.unmask_multiple(editor.template('What does the <mask> say about {thing}?', thing=quran))])

['Bible', 'book', 'report', 'survey', 'poll', 'study', 'Constitution', 'data', 'bible', 'election', 'census', 'law', 'UN', 'Quran', 'evidence', 'science', 'constitution', 'Koran', 'vote', 'bill', 'film', 'Holocaust', 'research', 'Pope', 'letter', 'article', 'verdict', 'world', 'movie', 'government', 'video', 'Torah', 'song', 'moon', 'public', 'dictionary', 'president', 'referendum', 'ACA', 'eclipse', 'church', 'US', 'media', 'Church', 'pope', 'record', 'test', 'answer', 'literature', 'Prophet', 'CDC', 'FDA', 'WHO', 'scripture', 'community', 'Buddha', 'ban', 'prophet', 'statement', 'interview', 'Gospel', 'Book', 'past', 'question', 'decision']


In [484]:
books = ['Bible', 'Constitution', 'Quran', 'Pope', 'Torah', 'Church', 'Buddha', 'Prophet', 'Gospel', 'Book']
data = editor.template(('What does the {book1} say about {thing}?', 'What does the {book2} say about {thing}?'),
                thing=quran, book1=books, book2=books, remove_duplicates=True)
test = Mft(data, labels=0)
test.run(pp, overwrite=True)
test.results.passed.mean()

0.9111111111111111

In [485]:
for f in np.where(test.results.passed == False)[0]:
    print(test.results.confs[f][1].round(3), test.data[f])

0.634 ('What does the Bible say about homosexuality?', 'What does the Church say about homosexuality?')
0.93 ('What does the Bible say about homosexuality?', 'What does the Prophet say about homosexuality?')
0.978 ('What does the Bible say about homosexuality?', 'What does the Gospel say about homosexuality?')
0.75 ('What does the Bible say about you?', 'What does the Church say about you?')
0.973 ('What does the Bible say about you?', 'What does the Gospel say about you?')
0.641 ('What does the Bible say about women?', 'What does the Torah say about women?')
0.87 ('What does the Bible say about women?', 'What does the Church say about women?')
0.931 ('What does the Bible say about women?', 'What does the Prophet say about women?')
0.987 ('What does the Bible say about women?', 'What does the Gospel say about women?')
0.56 ('What does the Bible say about women?', 'What does the Book say about women?')
0.958 ('What does the Bible say about us?', 'What does the Gospel say about us?')
0.6

### Inv, dirs

In [486]:
spacy_map =  pickle.load(open('/home/marcotcr/tmp/processed_qqp.pkl', 'rb'))

In [488]:
processed_qs = [spacy_map[x[0]] for x in qs] + [spacy_map[x[1]] for x in qs]

In [497]:
processed_pairs = [(spacy_map[x[0]],spacy_map[x[1]]) for x in qs]

In [532]:
editor.lexicons

{'male': ['John', 'Luke', 'Mark'], 'female': ['Mary', 'Judy', 'Julia']}

In [569]:
def change_names(pair):
    x0, x1 = pair
    ents_0 = set([x[0].text for x in x0.ents if x[0].ent_type_ == 'PERSON'])
    ents_1 = set([x[0].text for x in x1.ents if x[0].ent_type_ == 'PERSON'])
    ret = []
    if ents_0 and ents_0.intersection(ents_1):
        for e in ents_0.intersection(ents_1):
            if e == 'Quora':
                continue
            ret.extend([(pair[0].text.replace(e, n), pair[1].text.replace(e, n)) for n in editor.lexicons['male'] + editor.lexicons['female']])
    if ret:
        return ret
# change_names(s)

In [577]:
idxs = np.random.choice(len(processed_pairs), 100)
sl = [processed_pairs[i] for i in idxs]
data = Perturb.perturb(processed_pairs, change_names, keep_original=True)
# test = Dir(data, Expect.wrap(Expect.all(Expect.pairwise_to_group(Expect.monotonic_label(1, increasing=True, tolerance=0)))) )
test = Inv(data, threshold=0.1)
test.run(pp, overwrite=True)
1 - test.results.passed.mean()

0.1115196078431373

In [581]:
idxs = np.where(test.results.passed == False)[0]
for f in np.random.choice(idxs, 5):
    dataconfs = list(zip(test.data[f], test.results.confs[f][:, 1]))
    dataconfs[1:] = sorted(dataconfs[1:], key=lambda x: np.abs(x[1] - dataconfs[0][1]), reverse=True)
    print('\n'.join(['%.2f %s' % (x[1], x[0]) for x in dataconfs][:3]))
    print()

0.01 ('If Donald Trump becomes president, will he be a worse president than George W. Bush, equally bad, slightly better, or become as popular as Reagan?', 'If elected president, would Donald Trump do more damage to the U.S. than George W. Bush did?')
0.95 ('If Mary Trump becomes president, will he be a worse president than George W. Bush, equally bad, slightly better, or become as popular as Reagan?', 'If elected president, would Mary Trump do more damage to the U.S. than George W. Bush did?')
0.87 ('If Judy Trump becomes president, will he be a worse president than George W. Bush, equally bad, slightly better, or become as popular as Reagan?', 'If elected president, would Judy Trump do more damage to the U.S. than George W. Bush did?')

0.28 ('Who is Brian Bi and what does he do?', 'Who is Brian Bi?')
0.86 ('Who is Luke Bi and what does he do?', 'Who is Luke Bi?')
0.67 ('Who is John Bi and what does he do?', 'Who is John Bi?')

0.42 ('How do I play Clash of Clans on Windows Phone?', 

In [582]:
def change_name_in_one(pair):
    x0, x1 = pair
    ents_0 = set([x[0].text for x in x0.ents if x[0].ent_type_ == 'PERSON'])
    ents_1 = set([x[0].text for x in x1.ents if x[0].ent_type_ == 'PERSON'])
    ret = []
    if ents_0 and ents_0.intersection(ents_1):
        for e in ents_0.intersection(ents_1):
            if e == 'Quora':
                continue
            ret.extend([(pair[0].text.replace(e, n), pair[1].text) for n in editor.lexicons['male'] + editor.lexicons['female']])
            ret.extend([(pair[0].text, pair[1].text.replace(e, n)) for n in editor.lexicons['male'] + editor.lexicons['female']])
    if ret:
        return ret
# change_names(s)

In [648]:
monotonic = Expect.wrap(Expect.all(Expect.pairwise_to_group(Expect.monotonic_label(1, increasing=False, tolerance=0.1))))
idxs = np.random.choice(len(processed_pairs), 3000)
sl = [processed_pairs[i] for i in idxs]
data = Perturb.perturb(processed_pairs, change_name_in_one, keep_original=True)
test = Dir(data, monotonic) )
test.run(pp, overwrite=True)
test.results.passed.mean()

In [650]:
test.set_expect(monotonic)

In [651]:
could_fail = [i for i, x in enumerate(test.results.confs) if x[0, 1] < 0.9]
test.results.passed[could_fail].mean()

0.9225589225589226

In [652]:
idxs = np.where(test.results.passed == False)[0]
for f in np.random.choice(idxs, 5):
    dataconfs = list(zip(test.data[f], test.results.confs[f][:, 1]))
    dataconfs[1:] = sorted(dataconfs[1:], key=lambda x: (x[1] - dataconfs[0][1]), reverse=True)
    print('\n'.join(['%.5f %s' % (x[1], x[0]) for x in dataconfs][:3]))
    print()

0.86218 ('Is Donald Trump going to be the next US President?', 'Why should Donald Trump or Hillary Clinton be the next president of the US?')
0.98278 ('Is Donald Trump going to be the next US President?', 'Why should Luke Trump or Hillary Clinton be the next president of the US?')
0.95678 ('Is Donald Trump going to be the next US President?', 'Why should Mark Trump or Hillary Clinton be the next president of the US?')

0.87022 ('Why is Manaphy angsty throughout in Pokemon Ranger and the Temple of the Sea?', 'Why is Manaphy annoying in Pokemon Ranger and The Temple of The Sea?')
0.97493 ('Why is Manaphy angsty throughout in Pokemon Ranger and the Temple of the Sea?', 'Why is Manaphy annoying in Mark Ranger and The Temple of The Sea?')
0.97234 ('Why is Manaphy angsty throughout in Pokemon Ranger and the Temple of the Sea?', 'Why is Manaphy annoying in John Ranger and The Temple of The Sea?')

0.49178 ('Who will win Hillary Clinton Or Donald Trump?', 'Who would you vote for: Donald Trump 

In [616]:
could_fail = [i for i, x in enumerate(test.results.preds) if x[0] == 1]
# test.results.passed[could_fail].mean()

In [620]:
test.labels = [0 for _ in test.data]

In [657]:
test.results.passed

array([False,  True,  True, ..., False,  True, False])

In [664]:
test.results.confs[0]

array([[0.02683236, 0.97316766],
       [0.02507993, 0.97492003],
       [0.04010293, 0.9598971 ],
       [0.0105105 , 0.98948938],
       [0.98023516, 0.01976482],
       [0.95916265, 0.04083734],
       [0.9927184 , 0.00728171],
       [0.01190639, 0.98809355],
       [0.00472121, 0.99527884],
       [0.00650083, 0.99349904],
       [0.10066198, 0.89933807],
       [0.89147604, 0.10852402],
       [0.75067484, 0.24932514]])

In [692]:
# TODO: ALL
expectation = Expect.wrap(Expect.all(Expect.single_to_group(Expect.eq), ignore_first=True))
test.set_expect(expectation)
nondup = [i for i, x in enumerate(test.results.preds) if x[0] == 0]
dup = [i for i, x in enumerate(test.results.preds) if x[0] == 1]
test.results.passed.mean(), test.results.passed[nondup].mean(), test.results.passed[dup].mean()

(0.6292892156862745, 0.958974358974359, 0.3274647887323944)

In [682]:
# dataconfs[0]

In [684]:
could_fail

[1,
 2,
 4,
 5,
 11,
 12,
 17,
 19,
 21,
 23,
 26,
 29,
 30,
 32,
 33,
 34,
 35,
 36,
 40,
 42,
 44,
 46,
 47,
 49,
 51,
 52,
 53,
 55,
 58,
 60,
 61,
 62,
 64,
 65,
 67,
 68,
 70,
 71,
 73,
 74,
 75,
 76,
 77,
 79,
 81,
 82,
 83,
 87,
 88,
 89,
 90,
 91,
 92,
 95,
 99,
 101,
 102,
 105,
 106,
 107,
 111,
 113,
 114,
 115,
 118,
 121,
 126,
 128,
 129,
 130,
 137,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 151,
 152,
 153,
 155,
 158,
 161,
 162,
 164,
 166,
 167,
 168,
 169,
 170,
 173,
 174,
 175,
 184,
 189,
 190,
 192,
 193,
 194,
 196,
 197,
 198,
 200,
 202,
 204,
 206,
 207,
 208,
 209,
 211,
 212,
 218,
 220,
 227,
 229,
 232,
 235,
 236,
 239,
 241,
 242,
 246,
 247,
 248,
 249,
 252,
 254,
 256,
 259,
 260,
 261,
 262,
 264,
 280,
 282,
 283,
 284,
 285,
 286,
 288,
 291,
 292,
 295,
 299,
 300,
 302,
 304,
 309,
 311,
 315,
 318,
 320,
 321,
 322,
 326,
 327,
 328,
 329,
 330,
 331,
 336,
 343,
 344,
 347,
 348,
 350,
 351,
 352,
 353,
 355,
 357,
 358,
 359,
 36

In [695]:
idxs = np.where(test.results.passed == False)[0]
idxs = [x for x in idxs if x in nondup]
for f in np.random.choice(idxs, 5):
    dataconfs = list(zip(test.data[f], test.results.confs[f][:, 1]))
    dataconfs[1:] = sorted(dataconfs[1:], key=lambda x: (x[1]), reverse=True)
    print('\n'.join(['%.5f %s' % (x[1], x[0]) for x in dataconfs][:2]))
    print()

0.29917 ('What were the Jim Crow laws?', 'What were Jim Crow laws in the US? How did they get that name?')
0.86248 ('What were the Jim Crow laws?', 'What were Mark Crow laws in the US? How did they get that name?')

0.49603 ('Is Hillary Clinton a establishment politician?', 'Is Hillary Clinton a successful politician?')
0.69593 ('Is Hillary Clinton a establishment politician?', 'Is Mark Clinton a successful politician?')

0.37161 ("What would have happened if John Lennon hadn't been shot?", 'What if John Lennon were still alive?')
0.98960 ("What would have happened if John Lennon hadn't been shot?", 'What if Luke Lennon were still alive?')

0.47351 ('What do you feel about Donald Trump winning the elections?', 'How do you feel about Donald Trump winning the Republican nomination?')
0.99046 ('What do you feel about Donald Trump winning the elections?', 'How do you feel about Mary Trump winning the Republican nomination?')

0.48534 ('Why is Kangana Ranaut so overrated?', 'Do you think Ka

In [641]:
test.results.passed.mean()

0.4583333333333333

In [639]:
# could_fail

In [618]:
# len(could_fail)

In [596]:
# len(data)

In [600]:
idxs = np.where(test.results.passed == False)[0]
for f in np.random.choice(idxs, 5):
    dataconfs = list(zip(test.data[f], test.results.confs[f][:, 1]))
    dataconfs[1:] = sorted(dataconfs[1:], key=lambda x: (x[1] - dataconfs[0][1]), reverse=True)
    print('\n'.join(['%.5f %s' % (x[1], x[0]) for x in dataconfs][:3]))
    print()

0.00265 ('What is the Lewis structure of CH2O?', 'What is the Lewis structure of NO2Cl?')
0.16858 ('What is the Lewis structure of CH2O?', 'What is the John structure of NO2Cl?')
0.11668 ('What is the Lewis structure of CH2O?', 'What is the Mark structure of NO2Cl?')

0.49603 ('Is Hillary Clinton a establishment politician?', 'Is Hillary Clinton a successful politician?')
0.69593 ('Is Hillary Clinton a establishment politician?', 'Is Mark Clinton a successful politician?')
0.41251 ('Is Hillary Clinton a establishment politician?', 'Is Julia Clinton a successful politician?')

0.02236 ('What is it like to be interviewed for a job by Bill Gates?', 'How is it to be interviewed for a job by Bill Gates or Mark Zuckerberg?')
0.12260 ('What is it like to be interviewed for a job by Mark Gates?', 'How is it to be interviewed for a job by Bill Gates or Mark Zuckerberg?')
0.00963 ('What is it like to be interviewed for a job by Bill Gates?', 'How is it to be interviewed for a job by John Gates o

In [587]:
data[3]

[("What will happen to the next Star Wars movies after Carrie Fisher's death?",
  "What will Carrie Fisher's death mean for the next Star Wars movies?"),
 ("What will happen to the next Star Wars movies after John Fisher's death?",
  "What will Carrie Fisher's death mean for the next Star Wars movies?"),
 ("What will happen to the next Star Wars movies after Luke Fisher's death?",
  "What will Carrie Fisher's death mean for the next Star Wars movies?"),
 ("What will happen to the next Star Wars movies after Mark Fisher's death?",
  "What will Carrie Fisher's death mean for the next Star Wars movies?"),
 ("What will happen to the next Star Wars movies after Mary Fisher's death?",
  "What will Carrie Fisher's death mean for the next Star Wars movies?"),
 ("What will happen to the next Star Wars movies after Judy Fisher's death?",
  "What will Carrie Fisher's death mean for the next Star Wars movies?"),
 ("What will happen to the next Star Wars movies after Julia Fisher's death?",
  "What

## Squad

In [696]:
from mltests import bert_squad_model

In [697]:
model = bert_squad_model.BertSquad()

In [710]:
# Just makes confidence=1 for every prediction
pp = PredictorWrapper.wrap_predict(model.predict_pairs)

In [709]:
model.predict_pairs([('Who is dumb?', 'Mary is somewhat dumb. John is not so dumb.')])

['Mary']

In [712]:
pp([('Who is dumb?', 'Mary is somewhat dumb. John is not so dumb.')])

(['Mary'], array([1.]))

In [717]:
c = pickle.load(open('/home/marcotcr/work/ml-tests/data/common.pkl', 'rb'))

In [718]:
c.keys()

dict_keys(['female_names', 'male_names', 'names', 'last_names', 'countries', 'nationalities', 'cities', 'sexuality_adjs', 'religions', 'religion_adjs', 'verbs_3ps', 'verbs_3pp', 'nouns', 'adjs'])

In [720]:
editor.lexicons['male'] = c['male_names']
editor.lexicons['female'] = c['female_names']

In [722]:
# editor.lexicons

In [728]:
verbs = [x[0][0] for x in tg.unmask('Luke told Mary that she should probably stop <mask>.')]
verbs = [x for x in verbs if 'ing' in x]
print(verbs)

['drinking', 'smoking', 'crying', 'reading', 'writing', 'worrying', 'eating', 'driving', 'listening', 'praying', 'talking', 'working', 'trying', 'calling', 'laughing']


In [743]:
labels = []
data, maps = editor.template(('Who is currently {verb}?', '{male} told {female} she should probably stop {verb}'), nsamples=100, verb=verbs, return_maps=True)
labels += [m['female'] for m in maps]
n, maps = editor.template(('Who is currently {verb}?', '{male} told {female} he should probably stop {verb}'), nsamples=100,verb=verbs, return_maps=True)
labels += [m['male'] for m in maps]
data += n
n, maps = editor.template(('Who is currently {verb}?', '{female} told {male} he should probably stop {verb}'), nsamples=100, verb=verbs, return_maps=True)
labels += [m['male'] for m in maps]
data += n
n, maps = editor.template(('Who is currently {verb}?', '{female} told {male} she should probably stop {verb}'), nsamples=100, verb=verbs, return_maps=True)
labels += [m['female'] for m in maps]
data += n

In [758]:
test = Mft(data, labels=labels)
test.run(pp)
test.results.passed.mean()

In [771]:
for f in np.where(test.results.passed == False)[0]:
    print(test.data[f][1])
    print(test.data[f][0], test.labels[f])
    print('Bert:', test.results.preds[f])
    print()

Henry told Cynthia she should probably stop calling
Who is currently calling? Cynthia
Bert: Henry told Cynthia

Ryan told Sarah she should probably stop driving
Who is currently driving? Sarah
Bert: Ryan

Jeremy told Brianna she should probably stop driving
Who is currently driving? Brianna
Bert: Jeremy

William told Chelsea she should probably stop calling
Who is currently calling? Chelsea
Bert: William told Chelsea

Robert told Allison she should probably stop working
Who is currently working? Allison
Bert: Robert

Derek told Brooke she should probably stop laughing
Who is currently laughing? Brooke
Bert: Derek

John told Shannon she should probably stop calling
Who is currently calling? Shannon
Bert: John

Jesse told Jamie she should probably stop calling
Who is currently calling? Jamie
Bert: Jesse

Joshua told Katherine she should probably stop talking
Who is currently talking? Katherine
Bert: Joshua

Scott told Ella she should probably stop drinking
Who is currently drinking? Ella

In [775]:
liked = [x[0][0] for x in tg.unmask('John was told by Luke that he really likes <mask>.', beam_size=100)]

In [783]:
labels = []
data, maps = editor.template(('Who really likes {liked}?', '{male1} was told by {male2} that {male2} really likes {liked}.'), male1=editor.lexicons['male'], male2=editor.lexicons['male'], nsamples=100, verb=verbs, liked=liked, return_maps=True, remove_duplicates=True)
labels += [m['male2'] for m in maps]

In [784]:
test = Mft(data, labels=labels)
test.run(pp)
test.results.passed.mean()

0.5858585858585859

In [785]:
for f in np.where(test.results.passed == False)[0]:
    print(test.data[f][1])
    print(test.data[f][0], test.labels[f])
    print('Bert:', test.results.preds[f])
    print()

Cody was told by Aaron that Aaron really likes Sam.
Who really likes Sam? Aaron
Bert: Cody

Carlos was told by Jesse that Jesse really likes John.
Who really likes John? Jesse
Bert: Jesse that Jesse

Adrian was told by Marcus that Marcus really likes Sarah.
Who really likes Sarah? Marcus
Bert: Marcus that Marcus

Jeffrey was told by Jared that Jared really likes Jason.
Who really likes Jason? Jared
Bert: Jeffrey

Samuel was told by Lucas that Lucas really likes Ashley.
Who really likes Ashley? Lucas
Bert: Lucas that Lucas

Jonathan was told by Cameron that Cameron really likes himself.
Who really likes himself? Cameron
Bert: Jonathan was told by Cameron

Jackson was told by Logan that Logan really likes Jack.
Who really likes Jack? Logan
Bert: Jackson

Lucas was told by James that James really likes Alexa.
Who really likes Alexa? James
Bert: Lucas

Anthony was told by Dustin that Dustin really likes Lauren.
Who really likes Lauren? Dustin
Bert: Dustin that Dustin

Luis was told by Edwa

In [714]:
pp([('Who really likes candy?', 'John was told by Luke that he really likes candy.')])

(['John'], array([1.]))

In [713]:
print([x[0][0] for x in tg.unmask_multiple(editor.template('What does the <mask> say about {thing}?', thing=quran))])

['Bible', 'book', 'report', 'survey', 'poll', 'study', 'Constitution', 'data', 'bible', 'election', 'census', 'law', 'UN', 'Quran', 'evidence', 'science', 'constitution', 'Koran', 'vote', 'bill', 'film', 'Holocaust', 'research', 'Pope', 'letter', 'article', 'verdict', 'world', 'movie', 'government', 'video', 'Torah', 'song', 'moon', 'public', 'dictionary', 'president', 'referendum', 'ACA', 'eclipse', 'church', 'US', 'media', 'Church', 'pope', 'record', 'test', 'answer', 'literature', 'Prophet', 'CDC', 'FDA', 'WHO', 'scripture', 'community', 'Buddha', 'ban', 'prophet', 'statement', 'interview', 'Gospel', 'Book', 'past', 'question', 'decision']


In [786]:
import json
def load_squad(fold='validation'):
    answers = []
    data = []
    ids = []
    files = {
        'validation': '/home/marcotcr/datasets/squad/dev-v1.1.json',
        'train': '/home/marcotcr//datasets/squad/train-v1.1.json',
        }
    f = json.load(open(files[fold]))
    for t in f['data']:
        for p in t['paragraphs']:
            context = p['context']
            for qa in p['qas']:
                data.append({'passage': context, 'question': qa['question'], 'id': qa['id']})
                answers.append(set([(x['text'], x['answer_start']) for x in qa['answers']]))
    return data, answers


In [787]:
data, answers =  load_squad()

In [788]:
pairs = [(x['question'], x['passage']) for x in data]

In [805]:
editor.lexicons['male'][:10]

['Michael',
 'Christopher',
 'Matthew',
 'David',
 'James',
 'John',
 'Joshua',
 'Daniel',
 'Joseph',
 'William']

In [807]:
import re

In [815]:
re.search(r'\b%s\b' % 'John', 'Johnny .')
re.sub(r'\b%s\b' % 'John','Mary', 'John .John. Johnny John')

'Mary .Mary. Johnny Mary'

In [910]:
def change_name(x):
    q, c = x
    in_p = set()
    not_in_p = set()
    for n in editor.lexicons['male'][:10]:
        if re.search(r'\b%s\b' % n, c):
            in_p.add(n)
        else:
            not_in_p.add(n)
    if not in_p:
        return None
    ret = []
    ret_add = []
    for p in in_p:
        for n in not_in_p:
            ret.append((re.sub(r'\b%s\b' % p, n, q), re.sub(r'\b%s\b' % p, n, c)))
            ret_add.append((p, n))
    return ret[:10], ret_add[:10]
            
    
    

In [835]:
editor.lexicons['male'][:10]

['Michael',
 'Christopher',
 'Matthew',
 'David',
 'James',
 'John',
 'Joshua',
 'Daniel',
 'Joseph',
 'William']

In [837]:
# [i for i, x in enumerate(pairs) if 'John' in x[1]]

In [869]:
def new_eq(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    if meta:
        p, n = meta
        orig_pred =  re.sub(r'\b%s\b' % p, n, orig_pred)
    return pred == orig_pred
    
tt = Expect.wrap(Expect.all(Expect.pairwise_to_group(new_eq), ignore_first=True))

In [911]:
data, meta = Perturb.perturb(pairs, change_name, returns_additional=True)

In [923]:
test = Inv(data, expect=tt, meta=meta)

In [924]:
test.run(pp)

Predicting 12679 examples


In [925]:
test.results.passed.mean()

0.9823293172690764

In [920]:
# len(idxs)

In [926]:
idxs = np.where(test.results.passed == False)[0]
for f in np.random.choice(idxs, min(5, len(idxs)), replace=False):
    d = test.data[f]
    print(d[0][1])
    print()
    print(d[0][0])
    print(test.results.preds[f][0])
    diffs = np.where(test.results.preds[f] != test.results.preds[f][0])[0]
    for d in diffs:
        print('r%s' % str(test.meta[f][d]))
        print(test.results.preds[f][d])
    print()
    print()
    print()

Charles Avison, the leading British composer of concertos in the 18th century, was born in Newcastle upon Tyne in 1709 and died there in 1770. Basil Hume, Archbishop of Westminster, was born in the city in 1923. Vice Admiral Cuthbert Collingwood, 1st Baron Collingwood, was born in the city. Ironmaster, metallurgist, and member of parliament Isaac Lowthian Bell was born in the city in 1816. Other notable people born in or associated with Newcastle include: engineer and industrialist Lord Armstrong, engineer and father of the modern steam railways George Stephenson, his son, also an engineer, Robert Stephenson, engineer and inventor of the steam turbine Sir Charles Parsons, inventor of the incandescent light bulb Sir Joseph Swan, modernist poet Basil Bunting, and Lord Chief Justice Peter Taylor. Portuguese writer Eça de Queiroz was a diplomat in Newcastle from late 1874 until April 1879—his most productive literary period. Former Prime Minister of Thailand Abhisit Vejjajiva, was born in 

In [877]:
test.meta[0]

[{},
 ('John', 'Joshua'),
 ('John', 'Matthew'),
 ('John', 'William'),
 ('John', 'Christopher'),
 ('John', 'Michael'),
 ('John', 'James'),
 ('John', 'David'),
 ('John', 'Daniel'),
 ('John', 'Joseph')]

In [876]:
test.results.preds[0]

array(['John Fox', 'Joshua Fox', 'Matthew Fox', 'William Fox',
       'Christopher Fox', 'Michael Fox', 'James Fox', 'David Fox',
       'Daniel Fox', 'Joseph Fox'], dtype='<U15')

In [864]:
# meta[0]

In [861]:
# data[0]

<function checklist.expect.Expect.all.<locals>.expect(xs, preds, confs, labels=None, meta=None)>

In [853]:
change_name(pairs[178])[1][0]

('John', 'Joshua')

In [802]:
[x for x in pairs if 'John' in x[1]]

671

In [521]:
processed_qs[20]

How is vanilla extract made?

In [500]:
editor.lexicons['male']

['John', 'Luke', 'Mark']

In [498]:
len(processed_pairs)

40430

In [496]:
processed_qs[0].ents[0][0].ent_type_

'NORP'

In [440]:
# editor.template('This is {bad}',  bad=['bad', 'great', 'awesome'], return_maps=True)

In [14]:

editor.template(('This is {bad}', 'This is not {bad}'),  bad=['bad', 'great', 'awesome'], return_maps=False)

([('This is bad', 'This is not bad'),
  ('This is great', 'This is not great'),
  ('This is awesome', 'This is not awesome')],
 [{'bad': 'bad'}, {'bad': 'great'}, {'bad': 'awesome'}])

In [15]:
editor.template({
    'bad': 'This is {bad}',
    'notbad': 'This is not {bad}'},  bad=['bad', 'great', 'awesome'], return_maps=False)

[{'bad': 'This is bad', 'notbad': 'This is not bad'},
 {'bad': 'This is great', 'notbad': 'This is not great'},
 {'bad': 'This is awesome', 'notbad': 'This is not awesome'}]

In [57]:
a = {'bad': 'This is {bad}',
    'notbad': ('this is not {bad}', 'this is quite {abad}')}
b = ({'bad': 'This is {bad}', 'notbad': 'This is not {bad}'}, 'This is quite {bad}')
c = 'This is quite {bad}.'

In [64]:
editor.lexicons

{'male': ['John', 'Luke', 'Mark'], 'female': ['Mary', 'Judy', 'Julia']}

In [93]:
editor.template('This is {bad}, {male}', bad=('terrible', 'bad'), nsamples=None)

['This is terrible, John',
 'This is bad, John',
 'This is terrible, Luke',
 'This is bad, Luke',
 'This is terrible, Mark',
 'This is bad, Mark']